In [1]:
import torch
import torch.nn as nn
from lib2to3.pgen2 import token
from tracemalloc import start
import numpy as np
import csv
import keras
from sklearn.utils import shuffle 
import time as tt
from tqdm import tqdm

In [2]:
np.random.seed(1)
device = 'cuda'
torch.set_default_tensor_type('torch.cuda.FloatTensor')

def runtime(starttime):
    ret = tt.time() - starttime
    return ret

def unison_shuffle(a, b):
    p = np.random.permutation(len(a))
    return a[p], b[p]

def load_pretrained_embeddings(filepath):
    print("Loading in pretrained GloVe embeddings from: {}".format(filepath))
    
    embeddings = {}
    with open(filepath, 'r', encoding='utf-8') as embedfile:
        for line in embedfile:
            split_line = line.split()
            token = split_line[0]
            embed = np.array(split_line[1:], dtype=np.float64)
            embeddings[token] = embed
    
    print("{} words loaded into embedding".format(len(embeddings)))
    
    return (embeddings)

def tokenize_csv(filename):
    sen=[]
    lab=[]

    with open(filename, mode='r',encoding='utf-8') as csvfile:
        tweetreader = csv.reader(csvfile)
        firstline=True
        for sentence, label in tweetreader:
            if firstline:
                firstline=False
                continue
            else:
                sen.append(keras.preprocessing.text.text_to_word_sequence(sentence.encode('ascii', 'ignore').decode('ascii')))
                if label=='sadness':
                    lab.append([0,0,0,0,1])
                elif label=='happiness':
                    lab.append([0,0,0,1,0])
                elif label=='no emotion':
                    lab.append([0,0,1,0,0])
                elif label=='fear':
                    lab.append([0,1,0,0,0])
                else:
                    lab.append([1,0,0,0,0])

    return (sen, lab)

def tokenize_manual(filename, label):
    '''
        00001 sadness 
        00010 happiness
        00100 no emotion
        01000 fear
        10000 anger
    '''

    sen=[]
    lab=[]
    with open(filename, mode='r', encoding='utf-8') as csvfile:
        treader=csv.reader(csvfile)
        firstline=True
        for i_d, t in treader:
            if firstline:
                firstline=False
                continue
            else:
                sen.append(keras.preprocessing.text.text_to_word_sequence(t.encode('ascii', 'ignore').decode('ascii')))
                lab.append(label)
    
    return (sen, lab)

def tweet_vectorize(tweet_label_in, glove_embeds):
    out_vectors = []

    tweet_list = tweet_label_in[0]
    label_list = tweet_label_in[1]

    for i in range(len(tweet_list)):
        curr_tweet = []
        while len(tweet_list[i]) > 0 and len(curr_tweet) < 50:
            curr_word = tweet_list[i].pop(0)
            if curr_word in glove_embeds:
                curr_tweet.append(glove_embeds[curr_word])
            else:
                curr_tweet.append(np.zeros(50))     # dimensionality of embedding vector
        if len(curr_tweet) < 50:
            while len(curr_tweet) < 50:
                curr_tweet.append(np.zeros(50))
        out_vectors.append(curr_tweet)
    
    return (out_vectors, label_list)

def split_train_val_test(input, labels, man_input, man_lab):

  input_shuff, labels_shuff = shuffle(input, labels)

  training_proportion = 0.8
  validation_proportion = 0.1
  num_train = int(len(input_shuff) * training_proportion)
  num_val = int(len(input_shuff) * validation_proportion)

  input_train, input_valid, input_test = input_shuff[:num_train], input_shuff[num_train:num_train+num_val], input_shuff[num_train+num_val:]
  label_train, label_valid, label_test = labels_shuff[:num_train], labels_shuff[num_train:num_train+num_val], labels_shuff[num_train+num_val:]

  input_test += man_input
  label_test += man_lab

  return input_train, input_valid, input_test, label_train, label_valid, label_test

def convert_to_tensors(input_train, input_valid, input_test, label_train, label_valid, label_test):
  input_train = torch.as_tensor(input_train)
  input_valid = torch.as_tensor(input_valid)
  input_test = torch.as_tensor(input_test)
  label_train = torch.as_tensor(label_train)
  label_valid = torch.as_tensor(label_valid)
  label_test = torch.as_tensor(label_test)

  return input_train, input_valid, input_test, label_train, label_valid, label_test

def accuracy_calc(predictions, labels):
    prediction = torch.argmax(predictions, dim=1)
    prediction = nn.functional.one_hot(prediction, num_classes=5)
    ret = torch.unique(prediction[prediction == labels], return_counts=True)
    if ret[0][-1] == 1: ret = float(ret[1][1]/len(labels))
    else: ret = 0.0
    return ret

# Model Definition
class model(nn.Module):
    def __init__(self, in_channels=50, out_channels=30, kernel_size=3, stride=1, hidden_dim=4, num_layers=4):
        super(model, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.convolution_layer1 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size, stride),      # IN: [batch_size, num_vecs=50, embed_dim=50], OUT: [batch_size, out_channel=30, convolved=48]
            nn.BatchNorm1d(out_channels),                                   # unchanged; in-place op
            nn.ReLU(),                                                      # unchanged; in-place ops
            nn.MaxPool1d(kernel_size))                                      # IN: [batch_size, out_channel=30, convolved=48], OUT: [batch_size, 30, floor(convolved/kernel_size)=16]

        self.bilstm = nn.LSTM(input_size=int((self.in_channels - self.kernel_size + 1)/self.kernel_size), hidden_size=hidden_dim, bidirectional=True)   # IN: [batch_size, 30, 16], OUT: [batch_size, 30, 8]

        self.lin1 = nn.Linear(out_channels, 1)
        self.lin2 = nn.Linear(8, 5)     # whatever the 2nd of bilstm out is

        self.softmax = nn.Softmax(dim=2)

    def forward(self, x):
        x = torch.tensor(x, dtype=torch.float32)
        x = self.convolution_layer1(x)
        x, (hn, cn) = self.bilstm(x)
        x = torch.transpose(x, 1, 2)
        x = self.lin1(x)
        x = torch.transpose(x, 1, 2)
        x = self.lin2(x)
        x = self.softmax(x)
        return(x)

# training loop
def training_loop(model_instance, loss, optimizer, epochs, batch_size, x_train, y_train):
  for epoch in range(epochs):
    prediction = model_instance(x_train)
    print(prediction.shape)
    loss_value = loss(prediction, y_train)
    loss_value.backward()
    optimizer.step()
    optimizer.zero_grad()


In [3]:
runstart = tt.time()
small_embeds = load_pretrained_embeddings("D:/OneDrive - University of Toronto/School/NSCI Y3/WINTER/ECE324/glove.6B/glove.6B.50d.txt")

Loading in pretrained GloVe embeddings from: D:/OneDrive - University of Toronto/School/NSCI Y3/WINTER/ECE324/glove.6B/glove.6B.50d.txt
400000 words loaded into embedding


In [4]:
training_set = [[], []]
for i in range(1, 6):
    train_tokens = tokenize_csv("D:/OneDrive - University of Toronto/School/NSCI Y3/WINTER/ECE324/tweet_emotion/tweets_labels/data{}.csv".format(str(i)))
    temp_vec = tweet_vectorize(train_tokens, small_embeds)
    training_set[0] = training_set[0] + temp_vec[0]
    training_set[1] = training_set[1] + temp_vec[1]
    print('Finished vectorizing csv number {}'.format(str(i)), "Dataset size: ", len(training_set[0]), len(training_set[1]))

manual_names = ['anger', 'fear', 'happy', 'sad']
manual_labels = [[1,0,0,0,0], [0,1,0,0,0], [0,0,0,1,0], [0,0,0,0,1]]

manual_set = [[], []]
for i in range(len(manual_labels)):
    manual_tokens = tokenize_manual("D:/OneDrive - University of Toronto/School/NSCI Y3/WINTER/ECE324/tweet_emotion/manually_labeled/data_{}.csv".format(manual_names[i]), manual_labels[i])
    temp_vec = tweet_vectorize(manual_tokens, small_embeds)
    manual_set[0] = manual_set[0] + temp_vec[0]
    manual_set[1] = manual_set[1] + temp_vec[1]

Finished vectorizing csv number 1 Dataset size:  28091 28091
Finished vectorizing csv number 2 Dataset size:  65180 65180
Finished vectorizing csv number 3 Dataset size:  104505 104505
Finished vectorizing csv number 4 Dataset size:  143588 143588
Finished vectorizing csv number 5 Dataset size:  207863 207863


In [5]:
x_train, x_valid, x_test, y_train, y_valid, y_test = split_train_val_test(training_set[0], training_set[1], manual_set[0], manual_set[1])
x_train, x_valid, x_test, y_train, y_valid, y_test = convert_to_tensors(x_train, x_valid, x_test, y_train, y_valid, y_test)

C:\Users\garyx\AppData\Local\Temp/ipykernel_14980/131696540.py:117: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  input_train = torch.as_tensor(input_train)


#### Initial Tested Parameters
Training params: epochs = 10, learning rate = 0.001, batch size = 10000, optimizer = Adam, loss = BCELoss

Model params: out_channels = 30, kernel_size = 3, stride = 1, hidden_dim = 4 (hidden_dim - and maybe everything else - affects bilstm out dimensions which need to match lin layers)

Train & Valid accuracy w/ params: 60.4906% & 60.3420%

In [6]:
start = tt.time()

y_train = torch.squeeze(y_train, dim=1)
y_valid = torch.squeeze(y_valid, dim=1)
y_test = torch.squeeze(y_test, dim=1)

model_instance = model()

# Training hyperparams
loss = nn.BCELoss()
optimizer = torch.optim.Adam(model_instance.parameters(), lr=0.001)
batch_size = 10000
num_epochs = 10

curr_iter = 0
for epoch in tqdm(range(int(num_epochs)), desc='Training Epochs'):
    train_acc = []
    valid_acc = []
    train_loss = []
    valid_loss = []

    for b in range(0, len(x_train), batch_size):
        x_batch = x_train[b:b+batch_size]
        y_batch = y_train[b:b+batch_size]

        prediction = model_instance(x_batch)
        prediction = torch.squeeze(prediction, dim=1)
        y_batch = y_batch.type(torch.cuda.FloatTensor)
        loss_value = loss(prediction, y_batch)
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_acc.append(accuracy_calc(prediction, y_batch))
        train_loss.append(loss_value.item())
        
    train_loss = np.asarray(train_loss)
    train_acc = np.asarray(train_acc)
    print("Train Loss: {:.4f}\t\t".format(np.mean(train_loss))) 
    print("Train Acc: {:.4f}%\t".format(100*np.mean(train_acc)))

    for b in range(0, len(x_valid), batch_size):
        x_batch = x_valid[b:b+batch_size]
        y_batch = y_valid[b:b+batch_size]

        prediction = model_instance(x_batch)
        prediction = torch.squeeze(prediction, dim=1)
        y_batch = y_batch.type(torch.cuda.FloatTensor)
        loss_value = loss(prediction, y_batch)

        valid_acc.append(accuracy_calc(prediction, y_batch))
        valid_loss.append(loss_value.item())

    valid_loss = np.asarray(valid_loss)
    valid_acc = np.asarray(valid_acc)
    print("Valid Loss: {:.4f}\t\t".format(np.mean(valid_loss))) 
    print("Valid Acc: {:.4f}%\t".format(100*np.mean(valid_acc)))

print("Total training runtime: {:2f}".format(runtime(start)))

Training Epochs:   0%|          | 0/10 [00:00<?, ?it/s]C:\Users\garyx\AppData\Local\Temp/ipykernel_14980/131696540.py:159: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)


Train Loss: 0.4814		
Train Acc: 45.6831%	


Training Epochs:  10%|█         | 1/10 [00:19<02:57, 19.71s/it]

Valid Loss: 0.4671		
Valid Acc: 60.0301%	
Train Loss: 0.4518		
Train Acc: 60.0749%	


Training Epochs:  20%|██        | 2/10 [00:37<02:28, 18.50s/it]

...

Valid Loss: 0.3581		
Valid Acc: 60.0334%	
Train Loss: 0.3553		
Train Acc: 60.2157%	


Training Epochs:  90%|█████████ | 9/10 [02:34<00:16, 16.48s/it]

Valid Loss: 0.3535		
Valid Acc: 60.1201%	
Train Loss: 0.3505		
Train Acc: 60.4906%	


Training Epochs: 100%|██████████| 10/10 [02:50<00:00, 17.07s/it]

Valid Loss: 0.3483		
Valid Acc: 60.3420%	
Total training runtime: 170.709361


In [7]:
# Test accuracy:
prediction = model_instance(x_test)
prediction = torch.squeeze(prediction, dim=1)
test_acc = accuracy_calc(prediction, y_test)
print("Test acc: {:4f}".format(100*test_acc))

C:\Users\garyx\AppData\Local\Temp/ipykernel_14980/131696540.py:159: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)


Test acc: 59.994340
